In [1]:
from modularity_aware_gae.evaluation import community_detection, link_prediction
from modularity_aware_gae.input_data import load_data, load_labels
from modularity_aware_gae.louvain import louvain_clustering
from modularity_aware_gae.model import  GCNModelAE, GCNModelVAE, LinearModelAE, LinearModelVAE
from modularity_aware_gae.optimizer import OptimizerAE, OptimizerVAE
from modularity_aware_gae.preprocessing import *
from modularity_aware_gae.sampling import get_distribution, node_sampling
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Parameters related to the input data
flags.DEFINE_string('dataset', 'citeseer', 'Graph dataset, among: cora, citeseer, wiki, celegans, email, polbooks, texas, wisconsin')

flags.DEFINE_boolean('features', False, 'Whether to include node features')


# Parameters related to the Modularity-Aware GAE/VGAE model to train

# 1/3 - General parameters associated with GAE/VGAE
flags.DEFINE_string('model', 'linear_vae', 'Model to train, among: gcn_ae, gcn_vae, \
                                            linear_ae, linear_vae')
flags.DEFINE_float('dropout', 0., 'Dropout rate')
flags.DEFINE_integer('iterations', 200, 'Number of iterations in training')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate (Adam)')
flags.DEFINE_integer('hidden', 32, 'Dimension of the GCN hidden layer')
flags.DEFINE_integer('dimension', 16, 'Dimension of the output layer, i.e., \
                                       dimension of the embedding space')

# 2/3 - Additional parameters, specific to Modularity-Aware models
flags.DEFINE_float('beta', 0.0, 'Beta hyperparameter of Mod.-Aware models')
flags.DEFINE_float('lamb', 0.0, 'Lambda hyperparameter of Mod.-Aware models')
flags.DEFINE_float('gamma', 1.0, 'Gamma hyperparameter of Mod.-Aware models')
flags.DEFINE_integer('s_reg', 2, 's hyperparameter of Mod.-Aware models')

# 3/3 - Additional parameters, aiming to improve scalability
flags.DEFINE_boolean('fastgae', False, 'Whether to use the FastGAE framework')
flags.DEFINE_integer('nb_node_samples', 1000, 'In FastGAE, number of nodes to \
                                               sample at each iteration, i.e., \
                                               size of decoded subgraph')
flags.DEFINE_string('measure', 'degree', 'In FastGAE, node importance measure used \
                                          for sampling: degree, core, or uniform')
flags.DEFINE_float('alpha', 1.0, 'alpha hyperparameter associated with the FastGAE sampling')
flags.DEFINE_boolean('replace', False, 'Sample nodes with or without replacement')
flags.DEFINE_boolean('simple', True, 'Use simpler (and faster) modularity in optimizers')


# Parameters related to the experimental setup
flags.DEFINE_string('task', 'task_2', 'task_1: pure community detection \
                                       task_2: joint link prediction and \
                                               community detection')
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 1., 'Proportion of edges in validation set \
                                    for the link prediction task')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      for the link prediction task')
flags.DEFINE_boolean('validation', False, 'Whether to compute validation \
                                           results at each iteration, for \
                                           the link prediction task')
flags.DEFINE_boolean('verbose', True, 'Whether to print all comments details')


# Introductory message
if FLAGS.verbose:
    introductory_message()


 
 
 
[MODULARITY-AWARE GRAPH AUTOENCODERS]
 
 
 

EXPERIMENTAL SETTING 

- Graph dataset: citeseer
- Mode name: linear_vae
- Number of models to train: 10
- Number of training iterations for each model: 200
- Learning rate: 0.01
- Dropout rate: 0.0
- Use of node features in the input layer: False
- Dimension of the output layer: 16
- lambda: 0.0
- beta: 0.0
- gamma: 1.0
- s: 2
- FastGAE: no 

Final embedding vectors will be evaluated on:
- Task 2, i.e., joint community detection and link prediction

 
 
 



In [3]:
# Initialize lists to collect final results
mean_ami = []
mean_ari = []

mean_roc = []
mean_ap = []

# Load data
if FLAGS.verbose:
    print("LOADING DATA\n")
    print("Loading the", FLAGS.dataset, "graph")
adj_init, features_init = load_data(FLAGS.dataset)
#labels = load_labels(adj_init.shape[0])
if FLAGS.verbose:
    print("- Number of nodes:", adj_init.shape[0])
    #print("- Number of communities:", len(np.unique(labels)))
    print("- Use of node features:", FLAGS.features)
    print("Done! \n \n \n \n")

LOADING DATA

Loading the citeseer graph
- Number of nodes: 3279
- Use of node features: False
Done! 
 
 
 



In [4]:
# We repeat the entire training+test process FLAGS.nb_run times
for i in range(FLAGS.nb_run):
    if FLAGS.verbose:
        print("EXPERIMENTS ON MODEL", i + 1, "/", FLAGS.nb_run, "\n")
        print("STEP 1/3 - PREPROCESSING STEPS \n")


    # Edge masking for Link Prediction:
    if FLAGS.task == 'task_2':
        # Compute Train/Validation/Test sets
        if FLAGS.verbose:
            print("Masking some edges from the training graph, for link prediction")
            print("(validation set:", FLAGS.prop_val, "% of edges - test set:",
                  FLAGS.prop_test, "% of edges)")
        #adj, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test, FLAGS.prop_val)
        adj, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        adj = adj_init

    # Compute the number of nodes
    num_nodes = adj.shape[0]

    # Preprocessing on node features
    if FLAGS.verbose:
        print("Preprocessing node features")
    if FLAGS.features:
        features = features_init
    else:
        # If features are not used, replace feature matrix by identity matrix
        features = sp.identity(num_nodes)
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]
    if FLAGS.verbose:
        print("Done! \n")


    # Community detection using Louvain, as a preprocessing step
    if FLAGS.verbose:
        print("Running the Louvain algorithm for community detection")
        print("as a preprocessing step for the encoder")
    # Get binary community matrix (adj_louvain_init[i,j] = 1 if nodes i and j are
    # in the same community) as well as number of communities found by Louvain
    adj_louvain_init, nb_communities_louvain, partition = louvain_clustering(adj, FLAGS.s_reg)
    if FLAGS.verbose:
        print("Done! Louvain has found", nb_communities_louvain, "communities \n")

    # FastGAE: node sampling for stochastic subgraph decoding - used when facing scalability issues
    if FLAGS.fastgae:
        if FLAGS.verbose:
            print("Preprocessing operations related to the FastGAE method")
        # Node-level p_i degree-based, core-based or uniform distribution
        node_distribution = get_distribution(FLAGS.measure, FLAGS.alpha, adj)
        # Node sampling for initializations
        sampled_nodes, adj_label, adj_sampled_sparse = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        sampled_nodes = np.array(range(FLAGS.nb_node_samples))


    # Placeholders
    if FLAGS.verbose:
        print('Setting up the model and the optimizer')
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_layer2': tf.sparse_placeholder(tf.float32), # Only used for 2-layer GCN encoders
        'degree_matrix': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape = ()),
        'sampled_nodes': tf.placeholder_with_default(sampled_nodes, shape = [FLAGS.nb_node_samples])
    }


    # Create model
    if FLAGS.model == 'linear_ae':
        # Linear Graph Autoencoder
        model = LinearModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'linear_vae':
        # Linear Graph Variational Autoencoder
        model = LinearModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    elif FLAGS.model == 'gcn_ae':
        # 2-layer GCN Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # 2-layer GCN Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    else:
        raise ValueError('Undefined model!')


    # Optimizer
    if FLAGS.fastgae:
        num_sampled = adj_sampled_sparse.shape[0]
        sum_sampled = adj_sampled_sparse.sum()
        pos_weight = float(num_sampled * num_sampled - sum_sampled) / sum_sampled
        norm = num_sampled * num_sampled / float((num_sampled * num_sampled - sum_sampled) * 2)
    else:
        pos_weight = float(num_nodes * num_nodes - adj.sum()) / adj.sum()
        norm = num_nodes * num_nodes / float((num_nodes * num_nodes - adj.sum()) * 2)

    if FLAGS.model in ('gcn_ae', 'linear_ae'):
        opt = OptimizerAE(preds = model.reconstructions,
                          labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                          degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                          num_nodes = num_nodes,
                          pos_weight = pos_weight,
                          norm = norm,
                          clusters_distance = model.clusters)
    else:
        opt = OptimizerVAE(preds = model.reconstructions,
                           labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                           degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                           model = model,
                           num_nodes = num_nodes,
                           pos_weight = pos_weight,
                           norm = norm,
                           clusters_distance = model.clusters)
    if FLAGS.verbose:
        print("Done! \n")


    # Symmetrically normalized "message passing" matrices
    if FLAGS.verbose:
        print("Preprocessing on message passing matrices")
    adj_norm = preprocess_graph(adj + FLAGS.lamb*adj_louvain_init)
    adj_norm_layer2 = preprocess_graph(adj)
    if not FLAGS.fastgae:
        adj_label = sparse_to_tuple(adj + sp.eye(num_nodes))
    if FLAGS.verbose:
        print("Done! \n")


    # Degree matrices
    deg_matrix, deg_matrix_init = preprocess_degree(adj, FLAGS.simple)


    # Initialize TF session
    if FLAGS.verbose:
        print("Initializing TF session")
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    if FLAGS.verbose:
        print("Done! \n")


    # Model training
    if FLAGS.verbose:
        print("STEP 2/3 - MODEL TRAINING \n")
        print("Starting training")

    for iter in range(FLAGS.iterations):

        # Flag to compute running time for each iteration
        t = time.time()

        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_norm_layer2, adj_label, features, deg_matrix, placeholders)
        if FLAGS.fastgae:
            # Update sampled subgraph and sampled Louvain matrix
            feed_dict.update({placeholders['sampled_nodes']: sampled_nodes})
            # New node sampling
            sampled_nodes, adj_label, adj_label_sparse, = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)

        # Weights update
        outs = sess.run([opt.opt_op, opt.cost, opt.cost_adj, opt.cost_mod],
                        feed_dict = feed_dict)

        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display information on the iteration
            print("Iteration:", '%04d' % (iter + 1), "Loss:", "{:.5f}".format(avg_cost),
                  "Time:", "{:.5f}".format(time.time() - t))

    if FLAGS.verbose:
        print("Done! \n")

    # Compute embedding vectors, for evaluation
    if FLAGS.verbose:
        print("STEP 3/3 - MODEL EVALUATION \n")
        print("Computing the final embedding vectors, for evaluation")
    emb = sess.run(model.z_mean, feed_dict = feed_dict)
    if FLAGS.verbose:
        print("Done! \n")

    # Test model: link prediction (classification edges/non-edges)

    if FLAGS.verbose:
        print("Testing: link prediction")
    # Get ROC and AP scores
    roc_score, ap_score = link_prediction(test_edges, test_edges_false, emb)
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)
    if FLAGS.verbose:
        print("Done! \n")

EXPERIMENTS ON MODEL 1 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 576 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.74409 Time: 0.67783
Iteration: 0002 Loss: 1.68983 Time: 0.37846
Iteration: 0003 Loss: 1.71099 Time: 0.35691
Iteration: 0004 Loss: 1.67968 Time: 0.37227
Iteration: 0005 Loss: 1.64290 Time: 0.35892
Iteration: 0006 Loss: 1.67207 Time: 0.35718
Iteration: 0007 Loss: 1.61486 Time: 0.36004
Iteration: 0008 Loss: 1.63826 Time: 0.36800
Iteration: 0009 Loss: 1.59279 Time: 0.36654
Iteration: 0010 Loss: 1.58538 Time: 0.37983
Iteration: 0011 L

Iteration: 0175 Loss: 0.60888 Time: 0.36147
Iteration: 0176 Loss: 0.60304 Time: 0.36570
Iteration: 0177 Loss: 0.60086 Time: 0.37528
Iteration: 0178 Loss: 0.60229 Time: 0.37998
Iteration: 0179 Loss: 0.59798 Time: 0.37478
Iteration: 0180 Loss: 0.59454 Time: 0.37516
Iteration: 0181 Loss: 0.59856 Time: 0.38490
Iteration: 0182 Loss: 0.59291 Time: 0.36461
Iteration: 0183 Loss: 0.59172 Time: 0.34797
Iteration: 0184 Loss: 0.58774 Time: 0.35877
Iteration: 0185 Loss: 0.58905 Time: 0.35826
Iteration: 0186 Loss: 0.59008 Time: 0.35123
Iteration: 0187 Loss: 0.59030 Time: 0.36304
Iteration: 0188 Loss: 0.58640 Time: 0.35198
Iteration: 0189 Loss: 0.58597 Time: 0.38190
Iteration: 0190 Loss: 0.58415 Time: 0.37216
Iteration: 0191 Loss: 0.58260 Time: 0.36863
Iteration: 0192 Loss: 0.58215 Time: 0.38294
Iteration: 0193 Loss: 0.58053 Time: 0.37213
Iteration: 0194 Loss: 0.57663 Time: 0.36838
Iteration: 0195 Loss: 0.57867 Time: 0.36049
Iteration: 0196 Loss: 0.58048 Time: 0.35698
Iteration: 0197 Loss: 0.57948 Ti

Iteration: 0146 Loss: 0.65486 Time: 0.35846
Iteration: 0147 Loss: 0.66143 Time: 0.35035
Iteration: 0148 Loss: 0.65077 Time: 0.34765
Iteration: 0149 Loss: 0.65498 Time: 0.36535
Iteration: 0150 Loss: 0.64618 Time: 0.34820
Iteration: 0151 Loss: 0.63906 Time: 0.35402
Iteration: 0152 Loss: 0.64497 Time: 0.34995
Iteration: 0153 Loss: 0.64650 Time: 0.34545
Iteration: 0154 Loss: 0.64174 Time: 0.34634
Iteration: 0155 Loss: 0.63881 Time: 0.36574
Iteration: 0156 Loss: 0.63211 Time: 0.34245
Iteration: 0157 Loss: 0.63448 Time: 0.35620
Iteration: 0158 Loss: 0.63190 Time: 0.34486
Iteration: 0159 Loss: 0.62969 Time: 0.34698
Iteration: 0160 Loss: 0.62628 Time: 0.35483
Iteration: 0161 Loss: 0.62314 Time: 0.34424
Iteration: 0162 Loss: 0.62205 Time: 0.34928
Iteration: 0163 Loss: 0.61956 Time: 0.34876
Iteration: 0164 Loss: 0.61954 Time: 0.35632
Iteration: 0165 Loss: 0.61819 Time: 0.35243
Iteration: 0166 Loss: 0.61376 Time: 0.36554
Iteration: 0167 Loss: 0.61313 Time: 0.34021
Iteration: 0168 Loss: 0.61909 Ti

Iteration: 0117 Loss: 0.76325 Time: 0.34649
Iteration: 0118 Loss: 0.76869 Time: 0.34067
Iteration: 0119 Loss: 0.75288 Time: 0.34720
Iteration: 0120 Loss: 0.74674 Time: 0.34164
Iteration: 0121 Loss: 0.75004 Time: 0.34409
Iteration: 0122 Loss: 0.74960 Time: 0.36207
Iteration: 0123 Loss: 0.74855 Time: 0.34571
Iteration: 0124 Loss: 0.74241 Time: 0.36299
Iteration: 0125 Loss: 0.73081 Time: 0.35845
Iteration: 0126 Loss: 0.73548 Time: 0.35261
Iteration: 0127 Loss: 0.72367 Time: 0.35815
Iteration: 0128 Loss: 0.73004 Time: 0.37167
Iteration: 0129 Loss: 0.72003 Time: 0.37263
Iteration: 0130 Loss: 0.71741 Time: 0.38428
Iteration: 0131 Loss: 0.70476 Time: 0.36445
Iteration: 0132 Loss: 0.70465 Time: 0.34140
Iteration: 0133 Loss: 0.70127 Time: 0.34530
Iteration: 0134 Loss: 0.69388 Time: 0.34429
Iteration: 0135 Loss: 0.69578 Time: 0.34268
Iteration: 0136 Loss: 0.69370 Time: 0.35022
Iteration: 0137 Loss: 0.68843 Time: 0.34423
Iteration: 0138 Loss: 0.68449 Time: 0.34372
Iteration: 0139 Loss: 0.68746 Ti

Iteration: 0088 Loss: 0.92596 Time: 0.37075
Iteration: 0089 Loss: 0.92527 Time: 0.37098
Iteration: 0090 Loss: 0.91830 Time: 0.37070
Iteration: 0091 Loss: 0.90122 Time: 0.36478
Iteration: 0092 Loss: 0.89794 Time: 0.37021
Iteration: 0093 Loss: 0.89154 Time: 0.37059
Iteration: 0094 Loss: 0.88468 Time: 0.35805
Iteration: 0095 Loss: 0.88615 Time: 0.36841
Iteration: 0096 Loss: 0.87813 Time: 0.35918
Iteration: 0097 Loss: 0.87660 Time: 0.37640
Iteration: 0098 Loss: 0.87312 Time: 0.36623
Iteration: 0099 Loss: 0.86962 Time: 0.36835
Iteration: 0100 Loss: 0.84728 Time: 0.35300
Iteration: 0101 Loss: 0.84130 Time: 0.33759
Iteration: 0102 Loss: 0.85490 Time: 0.34069
Iteration: 0103 Loss: 0.82786 Time: 0.36593
Iteration: 0104 Loss: 0.84087 Time: 0.35293
Iteration: 0105 Loss: 0.83721 Time: 0.34635
Iteration: 0106 Loss: 0.82522 Time: 0.35140
Iteration: 0107 Loss: 0.81089 Time: 0.33771
Iteration: 0108 Loss: 0.82170 Time: 0.35902
Iteration: 0109 Loss: 0.80516 Time: 0.36424
Iteration: 0110 Loss: 0.78974 Ti

...

Iteration: 0001 Loss: 1.73564 Time: 0.65100
Iteration: 0002 Loss: 1.72959 Time: 0.35179
Iteration: 0003 Loss: 1.68907 Time: 0.34437
Iteration: 0004 Loss: 1.68176 Time: 0.35416
Iteration: 0005 Loss: 1.68285 Time: 0.35448
Iteration: 0006 Loss: 1.65073 Time: 0.35030
Iteration: 0007 Loss: 1.65613 Time: 0.35547
Iteration: 0008 Loss: 1.64184 Time: 0.35550
Iteration: 0009 Loss: 1.61626 Time: 0.35971
Iteration: 0010 Loss: 1.58925 Time: 0.35030
Iteration: 0011 Loss: 1.57449 Time: 0.35097
Iteration: 0012 Loss: 1.55177 Time: 0.35003
Iteration: 0013 Loss: 1.52981 Time: 0.35502
Iteration: 0014 Loss: 1.51987 Time: 0.35492
Iteration: 0015 Loss: 1.49439 Time: 0.34342
Iteration: 0016 Loss: 1.52860 Time: 0.35695
Iteration: 0017 Loss: 1.47827 Time: 0.34587
Iteration: 0018 Loss: 1.47368 Time: 0.34407
Iteration: 0019 Loss: 1.46826 Time: 0.36601
Iteration: 0020 Loss: 1.44815 Time: 0.35200
Iteration: 0021 Loss: 1.44719 Time: 0.33769
Iteration: 0022 Loss: 1.43344 Time: 0.34475
Iteration: 0023 Loss: 1.44406 Ti

Iteration: 0188 Loss: 0.58410 Time: 0.36289
Iteration: 0189 Loss: 0.58348 Time: 0.36058
Iteration: 0190 Loss: 0.58592 Time: 0.34816
Iteration: 0191 Loss: 0.58278 Time: 0.34400
Iteration: 0192 Loss: 0.58269 Time: 0.36936
Iteration: 0193 Loss: 0.58575 Time: 0.34161
Iteration: 0194 Loss: 0.58079 Time: 0.35555
Iteration: 0195 Loss: 0.58166 Time: 0.34531
Iteration: 0196 Loss: 0.57925 Time: 0.34832
Iteration: 0197 Loss: 0.58041 Time: 0.35418
Iteration: 0198 Loss: 0.57920 Time: 0.36564
Iteration: 0199 Loss: 0.57934 Time: 0.37008
Iteration: 0200 Loss: 0.57371 Time: 0.37421
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 

EXPERIMENTS ON MODEL 8 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a prep

Iteration: 0159 Loss: 0.61970 Time: 0.33782
Iteration: 0160 Loss: 0.62578 Time: 0.34717
Iteration: 0161 Loss: 0.62798 Time: 0.35100
Iteration: 0162 Loss: 0.62673 Time: 0.34923
Iteration: 0163 Loss: 0.62216 Time: 0.34722
Iteration: 0164 Loss: 0.61913 Time: 0.34620
Iteration: 0165 Loss: 0.61383 Time: 0.35558
Iteration: 0166 Loss: 0.61689 Time: 0.34400
Iteration: 0167 Loss: 0.61081 Time: 0.36401
Iteration: 0168 Loss: 0.61215 Time: 0.35199
Iteration: 0169 Loss: 0.61630 Time: 0.37094
Iteration: 0170 Loss: 0.60848 Time: 0.36012
Iteration: 0171 Loss: 0.60802 Time: 0.35425
Iteration: 0172 Loss: 0.60645 Time: 0.34557
Iteration: 0173 Loss: 0.60784 Time: 0.34995
Iteration: 0174 Loss: 0.59909 Time: 0.37219
Iteration: 0175 Loss: 0.59667 Time: 0.35585
Iteration: 0176 Loss: 0.59562 Time: 0.35487
Iteration: 0177 Loss: 0.59674 Time: 0.34594
Iteration: 0178 Loss: 0.59547 Time: 0.36918
Iteration: 0179 Loss: 0.59377 Time: 0.34609
Iteration: 0180 Loss: 0.59733 Time: 0.35751
Iteration: 0181 Loss: 0.59683 Ti

Iteration: 0130 Loss: 0.70858 Time: 0.34554
Iteration: 0131 Loss: 0.69163 Time: 0.35579
Iteration: 0132 Loss: 0.70166 Time: 0.36141
Iteration: 0133 Loss: 0.70375 Time: 0.36267
Iteration: 0134 Loss: 0.69909 Time: 0.37371
Iteration: 0135 Loss: 0.68902 Time: 0.35315
Iteration: 0136 Loss: 0.69729 Time: 0.34764
Iteration: 0137 Loss: 0.68745 Time: 0.34899
Iteration: 0138 Loss: 0.68726 Time: 0.36005
Iteration: 0139 Loss: 0.67834 Time: 0.34639
Iteration: 0140 Loss: 0.67448 Time: 0.34751
Iteration: 0141 Loss: 0.67519 Time: 0.35449
Iteration: 0142 Loss: 0.68023 Time: 0.35147
Iteration: 0143 Loss: 0.67336 Time: 0.34575
Iteration: 0144 Loss: 0.65573 Time: 0.35010
Iteration: 0145 Loss: 0.65791 Time: 0.35282
Iteration: 0146 Loss: 0.66535 Time: 0.36534
Iteration: 0147 Loss: 0.65339 Time: 0.35686
Iteration: 0148 Loss: 0.65316 Time: 0.34357
Iteration: 0149 Loss: 0.64910 Time: 0.35417
Iteration: 0150 Loss: 0.64483 Time: 0.35942
Iteration: 0151 Loss: 0.64861 Time: 0.35772
Iteration: 0152 Loss: 0.64334 Ti

Iteration: 0101 Loss: 0.83710 Time: 0.34575
Iteration: 0102 Loss: 0.86237 Time: 0.36407
Iteration: 0103 Loss: 0.84140 Time: 0.35822
Iteration: 0104 Loss: 0.84054 Time: 0.35380
Iteration: 0105 Loss: 0.82938 Time: 0.35468
Iteration: 0106 Loss: 0.82458 Time: 0.35938
Iteration: 0107 Loss: 0.81133 Time: 0.35758
Iteration: 0108 Loss: 0.80476 Time: 0.34826
Iteration: 0109 Loss: 0.80761 Time: 0.34529
Iteration: 0110 Loss: 0.80136 Time: 0.34463
Iteration: 0111 Loss: 0.79658 Time: 0.36755
Iteration: 0112 Loss: 0.79042 Time: 0.35804
Iteration: 0113 Loss: 0.78275 Time: 0.34913
Iteration: 0114 Loss: 0.77881 Time: 0.34981
Iteration: 0115 Loss: 0.78271 Time: 0.35016
Iteration: 0116 Loss: 0.76952 Time: 0.35131
Iteration: 0117 Loss: 0.76839 Time: 0.35680
Iteration: 0118 Loss: 0.75967 Time: 0.36019
Iteration: 0119 Loss: 0.76536 Time: 0.34910
Iteration: 0120 Loss: 0.75565 Time: 0.36113
Iteration: 0121 Loss: 0.75159 Time: 0.35192
Iteration: 0122 Loss: 0.75368 Time: 0.34775
Iteration: 0123 Loss: 0.74120 Ti

In [6]:
# Report final results
print("FINAL RESULTS \n")

print('Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on', FLAGS.dataset)

print("All scores reported below are computed over the", FLAGS.nb_run, "run(s) \n")

print("Link prediction:\n")
print("Mean AUC score: ", np.mean(mean_roc))
print("Std of AUC scores: ", np.std(mean_roc), "\n")

print("Mean AP score: ", np.mean(mean_ap))
print("Std of AP scores: ", np.std(mean_ap), "\n \n")

FINAL RESULTS 

Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on citeseer
All scores reported below are computed over the 10 run(s) 

Link prediction:

Mean AUC score:  0.7899190918971138
Std of AUC scores:  0.013616774985648498 

Mean AP score:  0.8346534571022513
Std of AP scores:  0.010142090058210514 
 

